In [14]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import gradio as gd
load_dotenv()

False

In [125]:
def record_user_details(email:str,name="unknown",notes="not provided"):
  print("recorded")
  return {"response":"recorded"}

In [124]:
def record_unknown_question(question:str):
  print(question)
  return {"response":"recorded"}

In [126]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [19]:
BASE_URL="https://api.groq.com/openai/v1"
API_KEY= os.getenv("MODEL_API_KEY","")
MODEL_NAME = os.getenv("MODEL_NAME",'llama-3.3-70b-versatile')

In [21]:
llm = OpenAI(api_key=API_KEY, base_url=BASE_URL)

In [103]:
record_unknown_question ='simply answer irrelavant question please ask only related to me'

In [ ]:
# If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \

In [127]:
tools=[{"type": "function", "function": record_user_details_json},{"type": "function", "function": record_unknown_question_json}]

In [139]:
import json
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [128]:
linkedin=f'https://www.linkedin.com/in/sanjeet-prasad-verma-07-/'
summary='Sanjeet prasad verma is from Jharkhand and currently working in TCS.'

In [129]:
name="Sanjeet prasad verma"
system_prompt = f"You are acting as {name}. You are answering questions on behalf of {name}, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. and provide the response that you don't have any idea of it and please ask something related to me and career \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. \
\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n \
With this context, please chat with the user, always staying in character as {name}."

In [143]:
def llm_response(message:str, history)-> str:
  messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": message}]
  while True:
      response = llm.chat.completions.create(
        messages=messages,
        model = MODEL_NAME,
        tools=tools
        )
      finish_reason = response.choices[0].finish_reason

      if finish_reason=="tool_calls":
          message = response.choices[0].message
          tool_calls = message.tool_calls
          results = handle_tool_calls(tool_calls)
          messages.append(message)
          messages.extend(results)
      else:
        break
  return response.choices[0].message.content

In [144]:
# llm_response("tell me something about your project you have worked on")

In [145]:
gd.ChatInterface(fn=llm_response, type='messages').launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


* Running on public URL: https://a2b1dcd47d6247c34f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
